In [1]:
import sys
import os

# Add the directory to the search path
sys.path.append('../code')

In [2]:
from pre_processing import extract_main_text
from custom_types import Paper

In [3]:
print('current directory: ' + os.getcwd())
data_path='../data/demo_data/'
files = [f for f in os.listdir(data_path) if f.endswith('.pdf')]
print('demofiles:')
for file in files:
    print(file)

current directory: C:\Users\linda.hemmann\Documents\FullTextTagging\notebooks
demofiles:
1-s2.0-S0090825815002577-main.pdf
1-s2.0-S014341799790086X-main.pdf
1-s2.0-S0167814004005663-main.pdf
1-s2.0-S1525730414002630-main.pdf
paper0.pdf
paper1.pdf
paper2.pdf
paper4.pdf
PII_ S0196-9781(97)00323-9.pdf
PII_ S1368-8375(98)00013-X.pdf


# Test creating a paper object and filling the data

In [4]:
pdf_path = data_path+files[1]
paper = Paper(pdf_path = pdf_path)

In [12]:
#str(paper)

In [6]:
main_text = extract_main_text(pdf_path)

In [10]:
paper.add_text(text=main_text)